# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
!pip install SQLAlchemy

In [2]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet','averaged_perceptron_tagger'])

import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score, roc_auc_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\talha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\talha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\talha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator,TransformerMixin
import pickle


In [44]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
# engine.table_names()
df = pd.read_sql("SELECT * FROM messages", engine)
X = df['message'].values
y = df.iloc[:,3:].values


In [45]:
y.shape

(26216, 36)

### 2. Write a tokenization function to process your text data

In [27]:
def tokenize(text):
    tokens = word_tokenize(text)
    
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [28]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [46]:
pipeline = pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))
    ])
pipeline1 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])
pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(RandomForestClassifier(n_estimators=200,min_samples_split=3)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [47]:
# train classifier
pipeline2.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('count_vectorizer',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x0000020E1027C430>)),
                                                                 ('tfidf_transformer',
                                                                  TfidfTransformer())])),
                                                ('starting_verb_transformer',
                                                 StartingVerbExtractor())])),
                ('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_split=3,
                                                                        n_estimators=200)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [49]:
y_pred = pipeline2.predict(X_test)   

In [19]:
y_test.values

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [50]:
print(classification_report(y_test,y_pred,target_names=df.columns.values[4:]))

                        precision    recall  f1-score   support

               request       0.91      0.44      0.59      1124
                 offer       0.00      0.00      0.00        30
           aid_related       0.78      0.63      0.70      2722
          medical_help       0.79      0.04      0.07       505
      medical_products       0.75      0.05      0.10       329
     search_and_rescue       0.83      0.03      0.05       180
              security       1.00      0.02      0.03       116
              military       0.89      0.04      0.07       217
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.21      0.34       433
                  food       0.89      0.38      0.54       762
               shelter       0.89      0.24      0.38       616
              clothing       0.56      0.05      0.09       104
                 money       1.00      0.02      0.04       154
        missing_people       0.00      

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_cla

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
parameters = {
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3, 4],
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, n_jobs=6)

In [15]:
cv.fit(X_train, y_train)  

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000229D8BFB790>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=6,
             param_grid={'clf__estimator__min_samples_split': [2, 3, 4],
                         'clf__estimator__n_estimators': [50, 100, 200]})

In [16]:
cv.best_estimator_.steps

[('vect',
  CountVectorizer(tokenizer=<function tokenize at 0x00000229D8BFB790>)),
 ('tfidf', TfidfTransformer()),
 ('clf',
  MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_split=3,
                                                         n_estimators=200)))]

In [17]:
y_pred_cv = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
print(classification_report(y_test,y_pred_cv,target_names=y.columns.values))

                        precision    recall  f1-score   support

               request       0.89      0.45      0.59      1084
                 offer       0.00      0.00      0.00        23
           aid_related       0.80      0.63      0.71      2718
          medical_help       0.70      0.04      0.07       558
      medical_products       0.72      0.05      0.10       333
     search_and_rescue       0.89      0.04      0.08       182
              security       0.00      0.00      0.00       122
              military       0.78      0.03      0.06       219
                 water       0.94      0.24      0.38       424
                  food       0.86      0.39      0.54       737
               shelter       0.83      0.27      0.41       568
              clothing       0.43      0.03      0.05       106
                 money       0.33      0.01      0.01       164
        missing_people       1.00      0.03      0.06        63
              refugees       0.40      

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [30]:
pipeline2.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('count_vectorizer',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x00000229D8BFB790>)),
                                                                 ('tfidf_transformer',
                                                                  TfidfTransformer())])),
                                                ('starting_verb_transformer',
                                                 StartingVerbExtractor())])),
                ('classifier',
                 MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [31]:
y_pred2 = pipeline2.predict(X_test)   

In [32]:
print(classification_report(y_test,y_pred2,target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      5031
               request       0.78      0.55      0.64      1103
                 offer       0.00      0.00      0.00        30
           aid_related       0.77      0.59      0.67      2714
          medical_help       0.52      0.23      0.32       516
      medical_products       0.65      0.30      0.41       333
     search_and_rescue       0.56      0.17      0.26       194
              security       0.20      0.03      0.05       132
              military       0.59      0.28      0.38       216
                 water       0.77      0.62      0.69       398
                  food       0.80      0.67      0.73       747
               shelter       0.77      0.54      0.64       554
              clothing       0.69      0.41      0.51       105
                 money       0.66      0.36      0.47       170
        missing_people       0.56      

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 9. Export your model as a pickle file

In [37]:
with open('classifier', 'ab') as file:
        pickle.dump(pipeline2, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.